In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
admis = pd.read_csv('../Data Files/NSS_Hospital_Inpatient_Admissions.csv',
                    infer_datetime_format=True, parse_dates=[8,9],
                   dtype= {6:'str',12:'str',14:'str',15:'str'})
visits = pd.read_csv('../Data Files/NSS_PrimaryCare_Office_Visits.csv',
                     infer_datetime_format=True, parse_dates=[12],
                    dtype={8:'str',9:'str',10:'str',11:'str'})
mem = pd.read_csv('../Data Files/NSS_GreatCare_Members.csv',
                  infer_datetime_format=True, parse_dates=[5,61,62],
                 dtype= {6:'str',10:'str',11:'str',13:'str'})
npi = pd.read_csv('../output/npi.csv')
npi = npi.drop(columns=['Unnamed: 0'])
print(admis.shape)
print(visits.shape)
print(mem.shape)
print(npi.shape)

C:\Users\pfbab\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(6213, 23)
(231373, 18)
(61519, 63)
(5415, 3)


In [3]:
visits = pd.merge(visits, npi, how='left', left_on='Service Provider NPI',right_on='npi' ).drop(columns=['entry','npi']).rename(columns={'enumeration':'Service Provider NPI Type'})
visits = pd.merge(visits, npi, how='left', left_on='Provider Group NPI',right_on='npi' ).drop(columns=['entry','npi']).rename(columns={'enumeration':'Provider Group NPI Type'})
admis = pd.merge(admis, npi, how='left', left_on='Attending Provider NPI',right_on='npi' ).drop(columns=['entry','npi']).rename(columns={'enumeration':'Attending Provider NPI Type'})
admis = pd.merge(admis, npi, how='left', left_on='Facility Provider NPI',right_on='npi' ).drop(columns=['entry','npi']).rename(columns={'enumeration':'Facility Provider NPI Type'})
visits.loc[visits['Provider Group NPI'].isna(),['Provider Group NPI Type']] = 'NPI-1'

In [4]:
pcp_visit = 0
readmit = 0
#pull patient numbers that have more then one admission to loop through
for patnum in admis['Patient ID'].drop_duplicates():
    window = pd.to_datetime(0) #set window as a datetime to make first loop work
    #create dataframe for each patient number keeping only the final discharge per Start Date
    patient = admis[admis['Patient ID']==patnum]
    patient = patient.sort_values(['Encounter Start Date','Encounter End Date']).drop_duplicates(subset=['Patient ID','Encounter End Date'], keep='first')
    patient = patient.sort_values('Encounter End Date').drop_duplicates(subset=['Patient ID','Encounter Start Date'], keep='last')
    vpatient = visits[visits['Patient ID']==patnum].sort_values('Encounter Date')
    dpatient = patient.set_index('Encounter End Date')['Encounter ID']
    dvpatient = vpatient.set_index('Encounter Date')['Encounter ID']
    visittimeline = pd.concat([dpatient ,dvpatient])
    visittimeline.sort_index(inplace=True)

    
    for rnum in range(0,len(patient)):
        if window < patient.iloc[rnum,8] :
            #test all admissions after rnum
            trigger = patient.iloc[rnum+1:,8]<=(patient.iloc[rnum,9] + pd.DateOffset(days=30)) #column 8=start, 9=end
            #add one if there are Trues
            readmit = readmit + int(trigger.sum() > 0)
            #save 30 day window date to remove dupicate dates
            window = (patient.iloc[rnum,9] + pd.DateOffset(days=30))
            if trigger.sum() > 0:
                admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'index_admis'] = True
                admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'readmission'] = True
            else:
                admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'index_admis'] = True
        else:
            admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'index_admis'] = False
    for rnum in range(0,len(visittimeline)-1):
        if (visittimeline[rnum].startswith('HIA') & visittimeline[rnum+1].startswith('COV'))&(visittimeline.index[rnum+1] <= (visittimeline.index[rnum])+pd.DateOffset(days=7)):
            pcp_visit += 1
            admis.loc[admis['Encounter ID']==visittimeline[rnum],'pcp_visit'] = True
            admis.loc[admis['Encounter ID']==visittimeline[rnum],'NPI_type'] = vpatient[vpatient['Encounter Date'].ge(visittimeline.keys()[rnum])].iloc[0]['Provider Group NPI Type']

        elif visittimeline[rnum].startswith('HIA'):
            admis.loc[admis['Encounter ID']==visittimeline[rnum],'pcp_visit'] = False
            try:
                admis.loc[admis['Encounter ID']==visittimeline[rnum],'NPI_type'] = vpatient[vpatient['Encounter Date'].ge(visittimeline.keys()[rnum])].iloc[0]['Provider Group NPI Type']
            except:
                #print(visittimeline[rnum:])
                admis.loc[admis['Encounter ID']==visittimeline[rnum],'NPI_type'] = 'None'

2013-04-01    HIA|DS1|10132|DS1|NPI|1386741635|1
2013-08-16    HIA|DS1|10132|DS1|NPI|1366545311|1
Name: Encounter ID, dtype: object
2010-06-06    HIA|DS1|10253|DS1|NPI|1396725677|1
2011-11-13    HIA|DS1|10253|DS1|NPI|1215987532|1
Name: Encounter ID, dtype: object
2014-04-02    HIA|DS1|10889|DS1|NPI|1427055839|1
2014-06-17    HIA|DS1|10889|DS1|NPI|1427055839|2
Name: Encounter ID, dtype: object
2014-01-19    HIA|DS1|11082|DS1|NPI|1508856535|1
2014-01-22    HIA|DS1|11082|DS1|NPI|1902803315|1
Name: Encounter ID, dtype: object
2014-09-16    HIA|DS1|11189|DS1|NPI|1033298617|1
2014-09-29    HIA|DS1|11189|DS1|NPI|1033298617|2
Name: Encounter ID, dtype: object
2012-10-26    HIA|DS1|11433|DS1|NPI|1033298617|1
2012-10-31    HIA|DS1|11433|DS1|NPI|1033298617|2
Name: Encounter ID, dtype: object
2017-09-16    HIA|DS1|11944|DS1|NPI|1548393127|1
2017-10-03    HIA|DS1|11944|DS1|NPI|1902803315|2
Name: Encounter ID, dtype: object
2018-03-18    HIA|DS1|12364|DS1|NPI|1184722779|3
2018-04-11    HIA|DS1|12364

2012-07-19     HIA|DS1|21926|DS1|NPI|1154428688|1
2012-08-03     HIA|DS1|21926|DS1|NPI|1689772592|1
2012-09-03     HIA|DS1|21926|DS1|NPI|1689772592|2
2012-09-23     HIA|DS1|21926|DS1|NPI|1689772592|3
2012-10-19     HIA|DS1|21926|DS1|NPI|1689772592|4
2012-10-31     HIA|DS1|21926|DS1|NPI|1689772592|5
2012-11-09     HIA|DS1|21926|DS1|NPI|1689772592|6
2012-12-05     HIA|DS1|21926|DS1|NPI|1689772592|7
2012-12-17     HIA|DS1|21926|DS1|NPI|1689772592|8
2012-12-26     HIA|DS1|21926|DS1|NPI|1689772592|9
2013-01-25    HIA|DS1|21926|DS1|NPI|1689772592|10
Name: Encounter ID, dtype: object
2012-08-03     HIA|DS1|21926|DS1|NPI|1689772592|1
2012-09-03     HIA|DS1|21926|DS1|NPI|1689772592|2
2012-09-23     HIA|DS1|21926|DS1|NPI|1689772592|3
2012-10-19     HIA|DS1|21926|DS1|NPI|1689772592|4
2012-10-31     HIA|DS1|21926|DS1|NPI|1689772592|5
2012-11-09     HIA|DS1|21926|DS1|NPI|1689772592|6
2012-12-05     HIA|DS1|21926|DS1|NPI|1689772592|7
2012-12-17     HIA|DS1|21926|DS1|NPI|1689772592|8
2012-12-26     H

2011-07-19    HIA|DS1|35624|DS1|NPI|1033298617|1
2011-08-21    HIA|DS1|35624|DS1|NPI|1033298617|2
2011-09-26    HIA|DS1|35624|DS1|NPI|1033298617|3
2011-11-01    HIA|DS1|35624|DS1|NPI|1174667158|1
Name: Encounter ID, dtype: object
2011-08-21    HIA|DS1|35624|DS1|NPI|1033298617|2
2011-09-26    HIA|DS1|35624|DS1|NPI|1033298617|3
2011-11-01    HIA|DS1|35624|DS1|NPI|1174667158|1
Name: Encounter ID, dtype: object
2011-09-26    HIA|DS1|35624|DS1|NPI|1033298617|3
2011-11-01    HIA|DS1|35624|DS1|NPI|1174667158|1
Name: Encounter ID, dtype: object
2013-05-05    HIA|DS1|35660|DS1|NPI|1184722779|1
2013-06-06    HIA|DS1|35660|DS1|NPI|1184722779|2
2013-06-19    HIA|DS1|35660|DS1|NPI|1184722779|3
2013-07-17    HIA|DS1|35660|DS1|NPI|1184722779|4
Name: Encounter ID, dtype: object
2013-06-06    HIA|DS1|35660|DS1|NPI|1184722779|2
2013-06-19    HIA|DS1|35660|DS1|NPI|1184722779|3
2013-07-17    HIA|DS1|35660|DS1|NPI|1184722779|4
Name: Encounter ID, dtype: object
2013-06-19    HIA|DS1|35660|DS1|NPI|1184722779

2016-04-22    HIA|DS1|49234|DS1|NPI|1184722779|1
2016-05-06    HIA|DS1|49234|DS1|NPI|1184722779|2
Name: Encounter ID, dtype: object
2012-10-08    HIA|DS1|49491|DS1|NPI|1710918545|1
2012-11-03    HIA|DS1|49491|DS1|NPI|1710918545|2
Name: Encounter ID, dtype: object
2012-03-23    HIA|DS1|49654|DS1|NPI|1689772592|1
2013-01-13    HIA|DS1|49654|DS1|NPI|1043502750|1
Name: Encounter ID, dtype: object
2014-11-28    HIA|DS1|50314|DS1|NPI|1033298617|1
2014-11-30    HIA|DS1|50314|DS1|NPI|1033298617|2
Name: Encounter ID, dtype: object
2014-12-19    HIA|DS1|5103|DS1|NPI|1235253816|1
2015-04-16    HIA|DS1|5103|DS1|NPI|1043502750|1
Name: Encounter ID, dtype: object
2015-12-03    HIA|DS1|51079|DS1|NPI|1033298617|1
2015-12-31    HIA|DS1|51079|DS1|NPI|1033298617|2
Name: Encounter ID, dtype: object
2014-01-18    HIA|DS1|51180|DS1|NPI|1033298617|1
2014-07-04    HIA|DS1|51180|DS1|NPI|1992818256|1
Name: Encounter ID, dtype: object
2016-09-15    HIA|DS1|51902|DS1|NPI|1902803315|1
2016-09-26    HIA|DS1|51902|D

In [5]:
admis['index_admis'].value_counts()

True     5536
False     613
Name: index_admis, dtype: int64

In [6]:
(admis['index_admis'].isna()).sum()

64

In [7]:
readmit

527

In [8]:
pcp_visit

1846

In [9]:
#question1
q1num = len(admis[(admis['index_admis']==True)&(admis['readmission']==True)])
q1denom = admis['index_admis'].value_counts()[True]
print('numerator: ' + str(q1num))
print('denominator: ' + str(q1denom))
print(q1num/q1denom)


numerator: 527
denominator: 5536
0.09519508670520231


In [10]:
#question2
q2num = admis['pcp_visit'].value_counts()[True]
q2denom = admis['index_admis'].value_counts().sum()
print('numerator: ' + str(q2num))
print('denominator: ' + str(q2denom))
print(q2num/q2denom)

numerator: 1846
denominator: 6149
0.30021141649048627


In [11]:
admis.NPI_type.value_counts()

NPI-2    4784
None      204
NPI-1     109
NPI-0       5
Name: NPI_type, dtype: int64

In [21]:
#question 3
q3num1I = len(admis[(admis['index_admis']==True)&(admis['readmission']==True)&(admis['NPI_type']=='NPI-1')])
q3denom1I = admis.loc[admis['NPI_type']=='NPI-1',['index_admis']].value_counts()[True]
q3num1G = len(admis[(admis['index_admis']==True)&(admis['readmission']==True)&(admis['NPI_type']=='NPI-2')])
q3denom1G = admis.loc[admis['NPI_type']=='NPI-2',['index_admis']].value_counts()[True]
q3num2I = admis.loc[admis['NPI_type']=='NPI-1',['pcp_visit']].value_counts()[True]
q3denom2I = admis.loc[admis['NPI_type']=='NPI-1',['index_admis']].value_counts().sum()
q3num2G = admis.loc[admis['NPI_type']=='NPI-2',['pcp_visit']].value_counts()[True]
q3denom2G = admis.loc[admis['NPI_type']=='NPI-2',['index_admis']].value_counts().sum()
print('Q1 Individual numerator: ' + str(q3num1I))
print('Q1 Individual denominator: ' + str(q3denom1I))
print(q3num1I/q3denom1I)
print('Q1 Group numerator: ' + str(q3num1G))
print('Q1 Group denominator: ' + str(q3denom1G))
print(q3num1G/q3denom1G)
print('Q2 Individual numerator: ' + str(q3num2I))
print('Q2 Individual denominator: ' + str(q3denom2I))
print(q3num1I/q3denom1I)
print('Q2 Group numerator: ' + str(q3num2G))
print('Q2 Group denominator: ' + str(q3denom2G))
print(q3num2G/q3denom2G)

Q1 Individual numerator: 8
Q1 Individual denominator: 96
0.08333333333333333
Q1 Group numerator: 417
Q1 Group denominator: 4317
0.0965948575399583
Q2 Individual numerator: 29
Q2 Individual denominator: 109
0.08333333333333333
Q2 Group numerator: 1816
Q2 Group denominator: 4784
0.3795986622073579


In [24]:
admis['NPI_type'].value_counts()

NPI-2    4784
None      204
NPI-1     109
NPI-0       5
Name: NPI_type, dtype: int64

In [26]:
admis.to_csv('../output/admis.csv')

In [27]:
admis['Facility Provider NPI'].value_counts()

1902803315    1204
1184722779    1196
1427055839     889
1689772592     679
1033298617     460
              ... 
1508856535       1
1083648810       1
1780614685       1
1033107214       1
1952312043       1
Name: Facility Provider NPI, Length: 220, dtype: int64